# Notebook to preprocess the data

In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:
import sys
sys.path.append('../../configs/')
sys.path.append('../../')

In [3]:
# with this line I have access to original headers as lists
from utils import load_train_datasets, load_test_datasets, print_logs

In [4]:
path2train = "../../DataHackaton/train/"
path2test = "../../DataHackaton/test/"
path2target = "../targets/"
path2features = "../features/"

## Read the data

In [5]:
candidates_df, vacants_df, application_df, app_stages_df, stages_df,_ = load_datasets(path2train)
candidates_test_df, vacants_test_df, application_test_df, stages_test_df,_ = load_datasets(path2test)

/home/davinchi/Documents/hackaton2020/venv/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
videosize_df = pd.read_csv(f"{path2features}videosize.csv").set_index("id")
# videosize_df_test = pd.read_csv(f"{path2features}videosize.csv").set_index("id")

# Creacion de la variable target

In [7]:
def create_target(path_to_target):
    # path_to_target : path to save the target variable

    app_stage = app_stages_df.join(stages_df, on="stage_id")
    app_stage["afin"] = (app_stage["status"] == "accepted") & ((app_stage["stage_type"]==0) |(app_stage["stage_type"]==1))
    app_stage["afin"] = app_stage["afin"].astype(int)
    app_stage = app_stage.loc[:,["application_id", "stage_id", "vacant_id", "afin"]].reset_index()
    app_stage = app_stage.rename(columns={"id": "app_stage_id"})

    app_id_afin = pd.DataFrame(app_stage.groupby("application_id")["afin"].apply(lambda a: any(a)))
    app_id_afin["afin"] = app_id_afin["afin"].astype(int)

    app_id_afin.to_csv(f"{path_to_target}app_id_afin.csv")
    print(f"target created at {path_to_target}app_id_afin.csv")
    return app_id_afin

# Creacion de la tabla de features

### 1. Creación de features en candidates

In [8]:
def create_features_cantidates(candidates_df, videosize=None, show_logs=False):
    # this function computes some features over the Candidates dataframe
    # it would also make a merge with videosize file if there is one
    print_logs("Computing Features on candidates: ...", show_logs)
    
    candidates_ft = candidates_df.copy()
    
    # 1. videosize
    if videosize is not None:
        candidates_ft = candidates_ft.merge(videosize, left_on="id", right_on="id")
        print_logs(f"computed videosize,  {candidates_ft.shape}", show_logs)
    
    # 2. longitud descripción
    candidates_ft["longitud_descripcion"] = candidates_ft["profile_description"].str.len()
    print_logs(f"computed longitud_descripcion  {candidates_ft.shape}", show_logs)
    
    # 3. Cantidad estudios
    candidates_ft["cantidad_estudios"] = candidates_ft["studies"].apply(lambda x: json.loads(x)).apply(len)
    print_logs(f"computed cantidad_estudios  {candidates_ft.shape}", show_logs)
    
    # 4. Cantidad experiencias
    candidates_ft["cantidad_experiences"] = candidates_ft["experiences"].apply(lambda x: json.loads(x)).apply(len)
    print_logs(f"computed cantidad_experiences  {candidates_ft.shape}", show_logs)
    
    # 5. birth year
    candidates_ft["birth_year"] = pd.to_datetime(candidates_ft["birthdate"], errors="coerce").dt.year
    print_logs(f"computed birth_year  {candidates_ft.shape}", show_logs)
    
    # 6. Availability to move
    candidates_ft["available_to_move"] = candidates_ft["available_to_move"].fillna(0).astype(int)
    print_logs(f"changed: available to move  {candidates_ft.shape}", show_logs)
    
    return candidates_ft

### 2. Creación de features en vacants

In [9]:
def create_features_vacants(vacants_df, feature_cols, show_logs=False):
    
    print_logs("Computing Features on vacants: ...", show_logs)
    
    vacants_ft = vacants_df.copy()
    vacants_ft["titles_and_studies"] = vacants_ft["titles_and_studies"].str.lower()
    
    vacants_ft = vacants_ft[feature_cols]
    print_logs(f"shape of vacants_features:  {vacants_ft.shape}", show_logs)
    
    return vacants_ft

### 3. Creación features mixtas

In [24]:
def create_features_mixtas(features):
    features["cumple_educacion"] = (features["education_level_vac"] == features["education_level"]).astype(int)
    features["cumple_salario"] = ((features["salary"] >= features["min_salary"]) & 
                                    (features["salary"] <= features["max_salary"])).astype(int)
    
    ordered_education = ['Básica primaria',
                     'Bachillerato (grados 6°, 7° u 8°)',
                     'Bachillerato (grados 9°, 10° y 11°)',
                     'Bachillerato completo',
                     'Técnico',
                     'Tecnólogo',
                     'Profesional',
                     'Especialización/ Maestría',
                     'Doctorado']
    
    features["education_level"] = pd.Categorical(features["education_level"],
                                            categories=ordered_education,
                                            ordered=True)

    features["education_level_vac"] = pd.Categorical(features["education_level_vac"],
                                                categories=ordered_education,
                                                ordered=True)
    features["education_level_number"] = features["education_level"].cat.codes.replace(-1, np.nan)
    features["education_level_vac_number"] = features["education_level_vac"].cat.codes.replace(-1, np.nan)

    features["education_difference"] = features["education_level_number"] - features["education_level_vac_number"]
    return features
    

In [ ]:
def create_dataset(train_or_test, cands, vacs, app, videosize):
    candidates_features = create_features_cantidates(cands, videosize, True)
    vacants_feature_cols = [
        "min_salary", 
        "max_salary", 
        "salary_type", 
        "education_level_vac",
        "experience_and_positions",
        "knowledge_and_skills",
        "titles_and_studies",
        "number_of_quotas"
    ]
    vacants_features = create_features_vacants(vacs, vacants_feature_cols, True)
    features = app[[ "vacant_id", "candidate_id"]].reset_index()

    # rename the id column to application_id
    features = features.rename(columns={"id": "application_id"})
    # merge the base of features with candidates and vacants
    features = features.merge(candidates_features, left_on= "candidate_id",  right_on="id")
    features = features.merge(vacants_features, left_on= "vacant_id",  right_on="id")
    features = create_features_mixtas(features)
    cols2delete = [
        "email", 
        "first_name", 
        "last_name", 
        "phone", 
        "profile_description", 
        "has_video",
        "studies",
        "experiences",
        "psy_tests",
        "identification_number",
        "country_birth",
        "birthdate",
        "civil_status",
        "title_or_profession"
    ]
    features= features.drop(cols2delete, axis=1)
    if train_or_test == "train":
        target = create_target(path2target)
        dataset = features.merge(target, left_on="application_id", right_on="application_id", how="right")
    else:
        dataset = features
        
    return dataset

In [ ]:
train_dataset = create_dataset("train", candidates_df, vacants_df, application_df, videosize_df)
test_dataset = create_dataset("test", candidates_test_df, vacants_test_df, application_test_df, videosize_test_df)

## Combine vacants and candidates features
* 1. Create features from vacants and candidates independently
* 2. Combine them with the applications dataframe by vacant_id and candidate_id as keys
* 3. Delete useless columns
* 4. Save features

In [25]:
candidates_features = create_features_cantidates(candidates_df, videosize_df, True)

Computing Features on candidates: ...
computed videosize,  (548364, 23)
computed longitud_descripcion  (548364, 24)
computed cantidad_estudios  (548364, 25)
computed cantidad_experiences  (548364, 26)
computed birth_year  (548364, 27)
changed: available to move  (548364, 27)


In [26]:
vacants_feature_cols = [
    "min_salary", 
    "max_salary", 
    "salary_type", 
    "education_level_vac",
    "experience_and_positions",
    "knowledge_and_skills",
    "titles_and_studies",
    "number_of_quotas"
]
vacants_features = create_features_vacants(vacants_df, vacants_feature_cols, True)

Computing Features on vacants: ...
shape of vacants_features:  (11693, 8)


In [27]:
features = application_df[[ "vacant_id", "candidate_id"]].reset_index()

# rename the id column to application_id
features = features.rename(columns={"id": "application_id"})

In [29]:
# merge the base of features with candidates and vacants
features = features.merge(candidates_features, left_on= "candidate_id",  right_on="id")
features = features.merge(vacants_features, left_on= "vacant_id",  right_on="id")

In [30]:
features = create_features_mixtas(features)

In [31]:
# features.head()

### delete some columns

In [32]:
cols2delete = [
    "email", 
    "first_name", 
    "last_name", 
    "phone", 
    "profile_description", 
    "has_video",
    "studies",
    "experiences",
    "psy_tests",
    "identification_number",
    "country_birth",
    "birthdate",
    "civil_status",
    "title_or_profession"
]
features= features.drop(cols2delete, axis=1)

### Save the features

In [33]:
features.to_csv(path2features+"features_on_vacants_and_candidates.csv", sep=";", index=False)

# Combine target and features

In [34]:
target_df = pd.read_csv(f"{path2target}app_id_afin.csv")

In [35]:
def combine_targets_and_features(targets, features):
    # this function will merge the features for every index
    # in targets dataframe. The key will be the application_id in
    # the features and targets dataframes
    
    dataset = features.merge(targets, left_on="application_id", right_on="application_id", how="right")
    return dataset

In [36]:
dataset = combine_targets_and_features(target_df, features)

In [37]:
# dataset.head()

### Save the dataset

In [38]:
dataset.to_csv(path2features+"dataset_feat_target.csv", sep=";", index= False)

In [39]:
dataset.to_pickle(path2features+"dataset_feat_target.pkl")